# Dedicated statistics and visualization

## 1. Figure 1 Configuration
Data for both panels of figure 1 is shown here, while the visualization is proceeded by Microsofit Excel and Sci2:
### 1.1 Figure 1(a) - Data preparation

In [ ]:
Records = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Complete_Records_WOS_20171111.csv', encoding = u'utf-8')
Records = Records.drop('Unnamed: 0', axis = 1)
Records = Records[Records.DT == 'Article']
PubYear = np.array(Records.PY)

#Extract Reprint author's nationality
Digremove = str.maketrans('0123456789',',,,,,,,,,,')
#Replace RP with C1 if RP is absent
Temp = []
for (index, item) in enumerate(Records.iterrows()):
    if item[1].RP != item[1].RP: #Incase the Reprint author is empty
        if item[1].C1 != item[1].C1: #Incase the author address is empty
            Temp.append(str(item[1].PA))
        else:
            Temp.append(str(item[1].C1).split('|')[0])
    else:
        Temp.append(item[1].RP)
        
RAU_Country = [str(author).lower().translate(Digremove).split(',')[-1].replace(' ','').replace('.','') for author in Temp]
RAU_Country = np.array(RAU_Country)
#Replace all double codes to usa
RAU_Country[[len(item)<3 for item in RAU_Country]] = 'usa'
RAU_Country[[item[-3:] == 'usa' for item in RAU_Country]] = 'usa'
Countries = pd.value_counts(RAU_Country).index

#Create Matrix
Year_Country = pd.DataFrame()
for topcountry in Countries:
    Year_Country.insert(0,topcountry,[(topcountry == country) for country in RAU_Country])
Year_Country.insert(0,'Year',PubYear)
Year_Country_Table = Year_Country.groupby('Year').sum()
Year_Country_Table.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Top20Country_PerYear_20171216.csv', encoding = u'utf-8')
#Output all countires
Year_Country_Table.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/AllCountry_PerYear_20180319.csv', encoding = u'utf-8')

### 1.2 Figure 1(b) - Data preparation

In [ ]:
Records = Records.assign(Country = RAU_Country)
#Take top 10 publication countries and count their top journals
country = Countries[10]
total = sum(pd.value_counts(Records[Records.Country == country].SO)[:5])
ToptenSO = pd.DataFrame(pd.value_counts(Records[Records.Country == country].SO)[:5]/total)

## 2. Figure 2 Configuration
This figure is composed by python, below we will show the data preparation and visualziation code:
### 2.1 Figure 2(a) & (b) - Data preparation
Concept Relative importance calculation:

In [ ]:
Records = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Complete_Records_WOS_20171111.csv', encoding = u'utf-8')
Records = Records.drop('Unnamed: 0', axis = 1)

#For Ti + Ab Please read this line
ConInPaper_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_Paper_TiAb_20171129.csv', false_values = ['0'], encoding = u'utf-8')
#For Ti only Please read this line
ConInPaper_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_Paper_Ti_20171216.csv', false_values = ['0'], encoding = u'utf-8')

ConInPaper_Matrix = ConInPaper_Matrix.drop('Unnamed: 0', axis = 1)
PubYear = np.array(Records.PY)
ConInPaper_Matrix.insert(0,'Year',PubYear)
del(Records)

#Per_Year_Total calculation -----New
Per_Year_Total = ConInPaper_Matrix.groupby('Year').sum()

#Concept Coverage
ConInPaper_Matrix = ConInPaper_Matrix.drop('Year', axis = 1)
Concept_Coverage = [False for i in range(len(ConInPaper_Matrix))]
for concept in ConInPaper_Matrix:
    Concept_Coverage = MatrixCombine(Concept_Coverage, ConInPaper_Matrix[concept])
print(sum(Concept_Coverage))
    
#For Ti + Ab output
Per_Year_Total.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_PerYear_TiAb_20171204.csv', encoding = u'utf-8')
#For Ti output
Per_Year_Total.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_PerYear_Ti_20171216.csv', encoding = u'utf-8')

#PerYear_RI calculation --------- New --------- Will raise changes in Per_Year_Total
temp = Per_Year_Total.T
for line in temp:
    temp[line] = temp[line] / sum(temp[line])
Per_Year_RI = temp.T

#For Ti+Ab output
Per_Year_RI.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_RI_TiAb_PerYear_20171205.csv', encoding = u'utf-8')
#For Ti output
Per_Year_RI.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_RI_Ti_PerYear_20171205.csv', encoding = u'utf-8')

#Reset the Per_Year_Total for latter use
Per_Year_Total = ConInPaper_Matrix.groupby('Year').sum()

Category Relative importancecalculation:

In [ ]:
ConInPaper_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_Paper_TiAb_20171129.csv', false_values = ['0'], encoding = u'utf-8')
#For Ti only Please read this line
ConInPaper_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_Paper_Ti_20171216.csv', false_values = ['0'], encoding = u'utf-8')


#Prepare the category dictionary
Categories = [item.split('|') for item in Waiting_List.Category]
for cate in Categories:
    try:
        cate.remove('')
    except:
        print('None')
Waiting_List.insert(0,'Categories',Categories)

temp = []
for item in Categories:
    temp += item
    
Cate_dict = {}
for cate in pd.value_counts(temp).index:
    Cate_dict.update({cate:[]})

for concept in Waiting_List.iterrows():
    for cate in concept[1].Categories:
        Cate_dict[cate].append(concept[1].Concepts)

#Generate Year_Category
Year_Category = pd.DataFrame()
for cate in Cate_dict:
    temp = [False for i in range(len(ConInPaper_Matrix))]
    print(cate)
    for concept in Cate_dict[cate]:
        temp = MatrixCombine(temp, ConInPaper_Matrix[concept])
    Year_Category.insert(0,cate,temp)
    
#For Ti + Ab output
Year_Category.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_Paper_TiAb_20171201.csv', encoding = u'utf-8')
#For Ti output
Year_Category.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_Paper_Ti_20171217.csv', encoding = u'utf-8')



#Category statistics
#For TI+AB
Category_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_Paper_TiAb_20171201.csv', false_values = ['0'], encoding = u'utf-8')
#For TI only
Category_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_Paper_Ti_20171217.csv', false_values = ['0'], encoding = u'utf-8')

Category_Matrix = Category_Matrix.drop('Unnamed: 0', axis = 1)
Records = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Complete_Records_WOS_20171111.csv', encoding = u'utf-8')
Records = Records.drop('Unnamed: 0', axis = 1)
PubYear = np.array(Records.PY)
del(Records)

#Per_Year_Total calculation -----New
Category_Matrix.insert(0,'Year',PubYear)
Per_Year_Total = Category_Matrix.groupby('Year').sum()

#For Ti + Ab
Per_Year_Total.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_PerYear_TiAb_20171204.csv', encoding = u'utf-8')
#For Ti only
Per_Year_Total.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_PerYear_Ti_20171217.csv', encoding = u'utf-8')


#PerYear_RI calculation --------- New --------- Will raise changes in Per_Year_Total
temp = Per_Year_Total.T
for line in temp:
    temp[line] = temp[line] / sum(temp[line])
Per_Year_RI = temp.T

#For Ti + Ab
Per_Year_RI.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_RI_PerYear_TiAb_20171205.csv', encoding = u'utf-8')
#For Ti only
Per_Year_RI.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_RI_PerYear_Ti_20171217.csv', encoding = u'utf-8')


### 2.2 Figure 2 - Visualziation
Both panels of Figure 2 used similar codez, packaged as a function later. Here we first showed the codez for Figure 2(a):

In [ ]:
ConcpetRI_data = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_RI_TiAb_PerYear_20171205.csv',
                            encoding = u'utf-8').set_index('Year').stack()
ConcpetRI_data.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_RIStacked_TiAb_PerYear_20171205.csv')

Paintdata = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Concept_RIStacked_TiAb_PerYear_20171205.csv',encoding = u'utf-8').sort_values(by = 'Cate')

Selected_Concepts = np.array(Paintdata.groupby('Cate').sum().sort_values('RI_value', ascending = False).index[:15])
Selected_Concepts[4] = 'Ecosystem'
Paintdata = Paintdata[[item in Selected_Concepts for item in Paintdata.Cate]]

%matplotlib qt5

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


sns.set(style="ticks")
#Set the RI to percentage according to the paper
Paintdata.RI_value *= 100
df = Paintdata
Con_fontsize = 12

# Initialize a grid of plots with an Axes for each walk
pal = sns.dark_palette("purple", n_colors = 15, reverse=True)
#pal = sns.cubehelix_palette(16, rot=-.25, light=.7)
grid = sns.FacetGrid(df, col="Cate", hue="Cate", col_wrap=5, size=2.5, palette = pal, legend_out = True)
#grid = sns.FacetGrid(df, row="Cate", hue="Cate", aspect = 1, size=5, palette = pal)
    
# Draw a line plot to show the trajectory of each random walk
#grid.map(plt.plot, "Year", "RI_value", marker="o", ms=4)
grid.map(sns.regplot,"Year","RI_value", line_kws = {"linewidth":0.5}, scatter_kws = {"marker":".", "s":10})
# Draw a horizontal line to show the starting point
grid.map(plt.axhline, y=0, ls=":", c=".5")
# Adjust the tick positions and labels
grid.set(xticks=np.arange(1915,2016,25), yticks=np.arange(-10,50,5),
         xlim=(1915, 2015), ylim=(-5, 20))

def label(x, color, label):
    ax = plt.gca()
    ax.text(0.5,1.05, label, fontsize = Con_fontsize, fontweight="bold", color=color, 
            ha="center", va="center", transform=ax.transAxes)

grid.map(label, "Cate")

# Adjust the arrangement of the plots
#grid.fig.subplots_adjust(wspace=.05, hspace=.05)

grid.set_xlabels("Year", fontsize = Con_fontsize, fontweight="bold")
grid.set_xticklabels(fontsize = Con_fontsize)

grid.set_ylabels("Relative Importance(%)", fontsize = Con_fontsize, fontweight="bold")
grid.set_yticklabels(fontsize = Con_fontsize)
grid.fig.tight_layout(w_pad=1)
grid.set_titles("")

The composition of Figure 2(b) is shown below:

In [ ]:
CateRITI_data = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_RI_PerYear_Ti_20171217.csv',
                            encoding = u'utf-8').set_index('Year').stack()
CateRITI_data.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/test.csv')
Paintdata = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Cate_Test.csv',encoding = u'utf-8').sort_values(by = 'Cate')

%matplotlib qt5

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


sns.set(style="ticks")
#Set the RI to percentage according to the paper
Paintdata.RI_value *= 100
df = Paintdata
Con_fontsize = 12

# Initialize a grid of plots with an Axes for each walk
pal = sns.dark_palette("purple", n_colors = 13, reverse=True)
#pal = sns.cubehelix_palette(13, start = 2.8, rot = .1, light = .7)
grid = sns.FacetGrid(df, col="Cate", hue="Cate", col_wrap = 5, size=2.5, palette = pal, legend_out = True)
#grid = sns.FacetGrid(df, row="Cate", hue="Cate", aspect = 1, size=5, palette = pal)
    
# Draw a line plot to show the trajectory of each random walk
#grid.map(plt.plot, "Year", "RI_value", marker="o", ms=4)
grid.map(sns.regplot,"Year","RI_value", line_kws = {"linewidth":0.5}, scatter_kws = {"marker":".", "s":10})
# Draw a horizontal line to show the starting point
grid.map(plt.axhline, y=0, ls=":", c=".5")
# Adjust the tick positions and labels
grid.set(xticks=np.arange(1915,2016,25), yticks=np.arange(-10,50,5),
         xlim=(1915, 2015), ylim=(-5, 40))

def label(x, color, label):
    ax = plt.gca()
    ax.text(0.5, 1.05, label, fontsize = Con_fontsize, fontweight="bold", color=color, 
            ha="center", va="center", transform=ax.transAxes)

grid.map(label, "Cate")

# Adjust the arrangement of the plots
#grid.fig.subplots_adjust(wspace=.05, hspace=.05)
grid.set_xlabels("Year", fontsize = Con_fontsize, fontweight="bold")
grid.set_xticklabels(fontsize = Con_fontsize)
grid.set_ylabels("Relative Importance(%)", fontsize = Con_fontsize, fontweight="bold")
grid.set_yticklabels(fontsize = Con_fontsize)
grid.fig.tight_layout(w_pad=1)
grid.set_titles("")

## 3. Figure 3 Configuration
We finished Figure 3 using both python and Adobe illustrator, here we'll show the codez:
### 3.1 Figure 3 - Data Preparation

Prepare the calculation of Preference Index:

In [ ]:
######Reload Category info
Category_Matrix = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Category_Paper_TiAb_20171201.csv', false_values = ['0'], encoding = u'utf-8')
Category_Matrix = Category_Matrix.drop('Unnamed: 0', axis = 1)
Category_Matrix.insert(0,'Country',RAU_Country)
Per_Country_Total = Category_Matrix.groupby('Country').sum()
Per_Country_Total.to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Country_Category_20180320.csv', encoding = u'utf-8')

#######Painting
PaintMatrix = Per_Country_Total.sort_values('Species', ascending = False)[:30].T.sort_values('usa', ascending = False)[:30]
#Proportion Calculation
for item in PaintMatrix:
    PaintMatrix[item] = PaintMatrix[item]/sum(PaintMatrix[item])

#Average Calculation
AveragePro = [np.mean(item[1]) for item in PaintMatrix.iterrows()]
HueAverage = pd.DataFrame()
for item in PaintMatrix:
    HueAverage.insert(0, item, AveragePro)
HueAverage = HueAverage.set_index(PaintMatrix.index)

#Calculate Residue of countries, showed up by change percentage
Residue = pd.DataFrame()
for item in PaintMatrix:
    Residue.insert(0, item, (PaintMatrix[item] - HueAverage[item]) / HueAverage[item])
Residue = Residue.set_index(PaintMatrix.index)

PaintMatrix.stack().to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Country_Top10_Concept_Fig4_20180323.csv', encoding = u'utf-8')
HueAverage.stack().to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Country_Top10_Concept_Average_20180323.csv', encoding = u'utf-8')
Residue.stack().to_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Country_Top10_Concept_Residue_20180326.csv', encoding = u'utf-8')

### 3.2 Figure 3 - Visualization

In [ ]:
Paintdata = pd.read_csv('D:/_Research/Project_Ecological_Development/Data_Processing/Country_Concept_Top30_Residue_20180326.csv', encoding = u'utf-8')
Country = Paintdata.Country
Concept = Paintdata.Concept
Paintdata.Percentage *= 100
Paintdata = Paintdata.pivot("Country", "Concept", "Percentage").T
#Paintdata = Paintdata.sort_values(by = "Species").T.sort_values(by = "USA")
%matplotlib qt5

sns.set()
sns.set(font = "Arial",font_scale = 1.5)
# Generate a custom diverging colormap or try "RdBu_r"
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Create a categorical palette to identify the networks
network_pal = sns.husl_palette(30, s = .45)
Concept_lut = dict(zip(Paintdata.index, network_pal))
Country_lut = dict(zip(Paintdata.T.index, network_pal))

# Convert the palette to vectors that will be drawn on the side of the matrix
Concept_colors = pd.Series(Paintdata.index, index = Paintdata.index).map(Concept_lut)
Country_colors = pd.Series(Paintdata.T.index, index = Paintdata.T.index).map(Country_lut)

ax = sns.clustermap(Paintdata, cmap = cmap, method = "average",
               row_colors=Concept_colors, col_colors=Country_colors,
               vmax = 120, vmin = -100, center = 0, linewidth = 1, figsize=(15, 15),
               cbar_kws = {"label":"Percentage Index(%)"})
plt.setp(ax.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(ax.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)

## 4. Figure 4 Configuration
We configured Figure 4 using Microsoft Excel and Adobe illustrator, the codez for data preparation is already shown in Section 1.1.